In [2]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [277]:
api_key = 
chat = ChatOpenAI(temperature=0.7, openai_api_key=api_key, model='gpt-3.5-turbo')



In [49]:
import chromadb
import pandas as pd
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=api_key)
embedding = [0.0] * 1536 
df = pd.read_csv('hidrogeno_verde.csv')
client = chromadb.Client()
collection = client.get_or_create_collection(name="hidrogeno")
df['date'] = pd.to_datetime(df['date'], format='%b %d, %Y @ %H:%M:%S.%f')
df['date'] = df['date'].dt.strftime('%Y-%m-%d')
df['combined'] = df['title'] + ' ' + df['text']



In [50]:

batch_size = 100

collection.upsert(
    documents=df['combined'].tolist(),
        metadatas=[{'media_outlet': row['media_outlet'], 'date': row['date'], 'title': row['title'], 'url': row['url']} for _, row in df.iterrows()],
    ids=[f'noticia_{i}' for i in range(len(df))],
)


In [51]:
import uuid

def store_in_chromadb(documents):
    for doc in documents:
        unique_id = str(uuid.uuid4())
        embedding = embeddings.embed_query(doc)
        collection.add(
            ids=[unique_id], 
            documents=[doc],
            embeddings=[embedding]
        )

In [115]:
import spacy
import ast
from collections import Counter

def count_mentions(array):
    try:
        array = ast.literal_eval(array)
        if not isinstance(array, list):
            raise ValueError("El input debe ser una lista representada como string.")
        nlp = spacy.load("es_core_news_md")
        documents = collection.get(ids=array)['documents']
        all_content = " ".join(documents)
        doc = nlp(all_content)
        personas = [ent.text for ent in doc.ents if ent.label_ == "PER"]
        conteo_personas = Counter(personas)
        resultado = "; ".join(
            f"{persona}, {frecuencia} veces" 
            for persona, frecuencia in conteo_personas.items()
        )
        return resultado
    except ValueError as ve:
        print(f"Error al procesar el array: {ve}")
        return "Error: El formato del array no es válido."
    except KeyError as ke:
        print(f"Error al obtener documentos: {ke}")
        return "Error: No se pudieron obtener los documentos."
    except Exception as e:
        print(f"Error inesperado: {e}")
        return "Error: Ocurrió un problema al contar las menciones."


In [223]:
import requests

import requests

def wikipedia_search(query):
    """Realiza una búsqueda en Wikipedia y maneja errores."""
    try:
        response = requests.get(
            "https://es.wikipedia.org/w/api.php",
            params={
                "action": "query",
                "format": "json",
                "titles": query,
                "prop": "extracts",
                "exintro": True
            },
            timeout=10  
        )
        response.raise_for_status()  
        pages = response.json().get('query', {}).get('pages', {})
        if not pages:
            return "No se encontró información en Wikipedia."
        
        page = next(iter(pages.values()))
        return page.get('extract', "No se encontró información en Wikipedia.")
    except requests.exceptions.RequestException as e:
        print(f"Error al realizar la solicitud a Wikipedia: {e}")
        return "No se pudo conectar a Wikipedia."
    except Exception as e:
        # Captura cualquier otro tipo de error
        print(f"Error inesperado: {e}")
        return "Ocurrió un error inesperado al buscar en Wikipedia."
    
    
def wikipedia_search_with_image(query):
    """Realiza una búsqueda en Wikipedia y obtiene el extracto y la imagen principal."""
    try:
        response = requests.get(
            "https://es.wikipedia.org/w/api.php",
            params={
                "action": "query",
                "format": "json",
                "titles": query,
                "prop": "extracts|pageimages",  #
                "exintro": True,  #
                "pithumbsize": 500  
            },
            timeout=10  
        )
        response.raise_for_status()  # Maneja errores HTTP
        pages = response.json().get('query', {}).get('pages', {})
        if not pages:
            return "No se encontró información en Wikipedia."

        page = next(iter(pages.values()))
        extract = page.get('extract', "No se encontró información en Wikipedia.")
        image_url = page.get('thumbnail', {}).get('source', "No se encontró una imagen principal.")
        
        return {
            "extract": extract,
            "image_url": image_url
        }
    except requests.exceptions.RequestException as e:
        print(f"Error al realizar la solicitud a Wikipedia: {e}")
        return "No se pudo conectar a Wikipedia."
    except Exception as e:
        print(f"Error inesperado: {e}")
        return "Ocurrió un error inesperado al buscar en Wikipedia."


In [ ]:
from difflib import SequenceMatcher
import spacy

nlp = spacy.load("es_core_news_md")

def simility(nombre1, nombre2, umbral=0.8):
    return SequenceMatcher(None, nombre1, nombre2).ratio() >= umbral

def get_news_from_name_chromadb(name):
    try:
        results = collection.query(
            query_texts=[name],
            n_results=5
        )
        
        filtered_results = []
        for i, document in enumerate(results["documents"][0]):
            doc = nlp(document)
            for ent in doc.ents:
                if ent.label_ == "PER" and simility(name.lower(), ent.text.lower()):
                    filtered_results.append(results["ids"][0][i])
                    break

        return filtered_results
    except Exception as e:
        print(f"Error en get_news_from_name_chromadb: {e}")
        return []


In [ ]:
from langchain.prompts import ChatPromptTemplate

def extract_person_data(texto):
    try:
        prompt_template = """   
            Del prompt entregado, con las siguientes keys: name, topic. En donde name es el nombre de la persona a la que se refiere 
            y topic el tema al que se refiere el prompt entregado.
                - El topic debe ser unicamente la palabra clave que engloba todo el tema
                - Si no existe persona a la que se refiera, deja solo como un string vacio
            prompt: {prompt}
        """
        template = ChatPromptTemplate.from_template(prompt_template)
        formatted_prompt = template.format(prompt=texto)
        response = llm.generate([formatted_prompt])
        if response and hasattr(response, 'generations') and response.generations:
            json_content = response.generations[0][0].text
            return json_content
        else:
            return None
    except Exception as e:
        print(f"Error al extraer datos: {e}")
        return None



In [212]:
from langchain.tools import Tool
unsplash_api_key = "ugvNkwuRRhHL-dgmmjIVb2ePmIOscTHMOSc8Ej0r6SA"
base_url = "https://api.unsplash.com/search/photos"

def search_images(query):
    params = {
        "query": query,
        "client_id": unsplash_api_key,
    }
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        image_urls = [result["urls"]["regular"] for result in data["results"]]
        return image_urls
    else:
        raise Exception(f"Error en la API de Unsplash: {response.status_code} - {response.text}")

In [205]:
from langchain_community.tools import DuckDuckGoSearchRun
import json

search_tool = DuckDuckGoSearchRun()
def search_in_web(data: str) -> str:
    try:
        info_web = search_tool.invoke(data)
        if info_web:
            return info_web 
        else:
            print("No se encontró información relevante en la web.")
    except Exception as e:
        print(f"Error al buscar en DuckDuckGo: {e}")
    return data


In [306]:
import json

def extract_personal_data_from_news_iteratively(data_list):
    try:
        initial_prompt_template = """
                Estructura la información proporcionada como un JSON con las claves:
                - 'news_id': array de ids de noticias.
                - 'name': nombre de la persona.
                Datos del texto:
                {prompt}
        """
        formatted_prompt = initial_prompt_template.format(prompt=data_list)
        
        response = llm.generate([formatted_prompt])
        
        # Verificar si hay generaciones disponibles
        if not response.generations or not response.generations[0]:
            return "Error: No se generaron respuestas en el primer paso."

        response_text = response.generations[0][0].text.strip()

        # Intentar cargar el JSON
        try:
            extracted_data = json.loads(response_text)
        except json.JSONDecodeError:
            return "Error: La respuesta no es un JSON válido."

        name = extracted_data.get('name')
        array = extracted_data.get('news_id', [])
        
        # Verificar si array está vacío
        if not array:
            return "Error: No se encontraron IDs de noticias en la respuesta."

        concatenated_results = ""
        for i, data in enumerate(data_list):
            if i >= len(array):
                concatenated_results += f"Advertencia: El índice {i} no tiene un ID correspondiente en 'array'.\n"
                continue
            
            try:
                documents = collection.get(ids=array)['documents'][i]
            except IndexError:
                concatenated_results += f"Advertencia: No se encontró un documento para el ID en el índice {i}.\n"
                continue

            all_content = truncate_text(" ".join(documents))

            # Segundo prompt para resumir la información
            summarization_prompt_template = """
                Resume la información personal de {name} encontrada en este texto.
                Incluye nombre completo, fecha de nacimiento, lugar de residencia, ocupación, logros y citas textuales relevantes.
                Si no hay información relevante, responde 'No se encontró información personal relevante.'
                Texto:
                {concatenated_text}
            """
            formatted_summarization_prompt = summarization_prompt_template.format(name=name, concatenated_text=all_content)

            summarization_response = llm.generate([formatted_summarization_prompt])

            # Verificar si hay generaciones disponibles
            if not summarization_response.generations or not summarization_response.generations[0]:
                concatenated_results += f"Advertencia: No se generó un resumen para el texto en el índice {i}.\n"
                continue

            summary = summarization_response.generations[0][0].text.strip()

            # Concatenar resultados
            concatenated_results += f"Texto {i+1}:\n{summary}\n\n"

        return concatenated_results

    except ValueError as ve:
        return f"Error de validación: {str(ve)}"
    except Exception as e:
        return f"Se produjo un error inesperado: {str(e)}"

def truncate_text(text, max_tokens=3000):
    tokens = text.split()
    if len(tokens) > max_tokens:
        return " ".join(tokens[:max_tokens])
    return text


In [247]:
from datetime import datetime
from langchain_core.tools import tool
import spacy
import json
from difflib import SequenceMatcher

nlp = spacy.load("es_core_news_md")


def generate_filters_from_prompt(prompt_texto):
    prompt = PromptTemplate(
        input_variables=["texto"],
        template=(
            "Del siguiente texto, extrae un JSON que contenga los campos: 'topic', 'start_date', y 'end_date'. "
            "El campo 'topic' debe ser una palabra o frase que describa el tema. "
            "Los campos 'start_date' y 'end_date' deben ser fechas en formato 'YYYY-MM-DD'. "
            "Si no se hace referencia a ninguna fecha, usa la menor fecha posible para 'start_date' (0001-01-01) "
            "y la mayor fecha posible para 'end_date' (9999-12-31). Texto: {texto}"
        )
    )
    
    formatted_summarization_prompt = prompt.format(texto=prompt_texto)
    summarization_response = llm.generate([formatted_summarization_prompt])
    response = summarization_response.generations[0][0].text.strip()

    try:
        parsed_json = json.loads(response)
        parsed_json["start_date"] = parsed_json.get("start_date", "0001-01-01")
        parsed_json["end_date"] = parsed_json.get("end_date", "9999-12-31")
        return json.dumps(parsed_json)
    except json.JSONDecodeError:
        return {"error": "El modelo no generó un JSON válido.", "output": response}
    

def search_with_filters(response_json):
    try:
        filters = generate_filters_from_prompt(response_json)
        response_json = json.loads(filters)
        tematica = response_json.get("topic", "")
        fecha_inicio = response_json.get("start_date", None)
        fecha_fin = response_json.get("end_date", None)
    except json.JSONDecodeError:
        return {"error": "Invalid JSON input"}

    def parse_date(date_str):
        try:
            return datetime.strptime(date_str, "%Y-%m-%d") if date_str and date_str != "N/A" else None
        except ValueError:
            return None

    fecha_inicio = parse_date(fecha_inicio)
    fecha_fin = parse_date(fecha_fin)

    results = collection.query(
        query_texts=[tematica],
        n_results=10,
    )

    all_dates = [
        datetime.strptime(metadata["date"], "%Y-%m-%d")
        for metadata in results["metadatas"][0]
        if "date" in metadata and metadata["date"] and metadata["date"] != "N/A"
    ]

    if not fecha_inicio and all_dates:
        fecha_inicio = min(all_dates)
    if not fecha_fin and all_dates:
        fecha_fin = max(all_dates)

    filtered_ids = []
    for i, metadata in enumerate(results["metadatas"][0]):
        if "date" in metadata and metadata["date"]:
            try:
                result_date = datetime.strptime(metadata["date"], "%Y-%m-%d")
                if (
                    (fecha_inicio is None or result_date >= fecha_inicio) and
                    (fecha_fin is None or result_date <= fecha_fin)
                ):
                    filtered_ids.append(results["ids"][0][i])
            except ValueError:
                continue  
    data = {}
    data['tematica'] = tematica
    data['start_date'] = fecha_inicio.isoformat()
    data['end_date'] = fecha_fin.isoformat()
    data['news_id']=filtered_ids
    return json.dumps(data)

In [320]:
from langchain_openai import OpenAI
from langchain.agents import Tool, initialize_agent
from langchain.agents import AgentType


tools = [

    Tool(
        name="count mentions",
        func=count_mentions,
        description="Calcula la cantidad total de menciones de personas en un conjunto de noticias dado un array de identificadores de noticias."
    ),
    Tool(
        name="Get user news ChromaDB",
        func=get_news_from_name_chromadb,
        description="Recupera un array con los identificadores de noticias en las que aparece una persona específica, dado su nombre."
    ),
    Tool(
        name="Extract personal data from news",
        func=extract_personal_data_from_news_iteratively,
        description="Procesa un array de identificadores de noticias (news_id) y un nombre de persona (name) para extraer información personal relevante de las noticias indicadas."
    ),
    Tool(
        name="Get web data",
        func=search_in_web,
        description="Realiza búsquedas específicas en la web para obtener información detallada o elementos relacionados con un tema en particular."
    ),
    Tool(
        name="Extract person data",
        func=extract_person_data,
        description="Genera un JSON que contiene el tema del que se habla (topic) y el nombre de la persona relevante (name), basado en un prompt base."
    ),
    Tool(
        name="Wikipedia image",
        func=wikipedia_search_with_image,
        description= " Busca imágenes relacionadas con un nombre o concepto específico en Wikipedia."
        
    ),
    Tool(
        name="Search data with filters",
        func=search_with_filters,
        description="En base a un tema entregado, y de forma opcional una fecha de incio y fin, obtiene noticias relacionadas a lo que se busca"
    ),
    Tool(
        name="Wikipedia Search",
        func=wikipedia_search,
        description="Busca información general en Wikipedia sobre el tema proporcionado."
    ),
]
# Definir el prompt usando PromptTemplate
prompt_template = """
Eres un agente capaz de interactuar con diversas herramientas para obtener información relevante y responder preguntas de manera precisa.  
Debes utilizar las herramientas de forma específica y únicamente cuando sean necesarias según el requerimiento de la pregunta. A continuación, se describe cada herramienta y sus usos:

1. **ChromaDB Search**  
   - Busca información específica en una base de datos de documentos relevantes.  
   - Úsala como primera opción si la información puede encontrarse en la base de datos interna.  

2. **Wikipedia Search**  
   - Realiza búsquedas generales en Wikipedia sobre un tema o nombre proporcionado.  
   - Úsala solo si no se encuentra la información en ChromaDB.  

3. **Count Mentions**  
   - Recibe un array de IDs de noticias en el formato `['noticia_389', 'noticia_350', ...]` y cuenta cuántas veces se mencionan personas en estas noticias.  
   - Úsala únicamente si la pregunta requiere contar menciones de personas.  

4. **Get User News ChromaDB**  
   - Recibe un nombre y devuelve un array de IDs de noticias relacionadas con esa persona.  
   - Úsala si se requiere identificar noticias vinculadas con un usuario.  

5. **Extract Person Data**  
   - Genera un JSON con la clave `topic` para el tema principal y `name` para el nombre de la persona involucrada.  
   - Úsala si se necesita identificar el tema de discusión y el nombre de la persona en cuestión.  

6. **Extract Personal Data from News**  
   - Recibe un `news_id` (array de IDs de noticias) y un `name` (nombre de la persona) en un string, y devuelve información relevante directamente extraída de las noticias proporcionadas.  
   - Responde estrictamente en el formato `news_id, name` sin incluir texto adicional.  

7. **Get Web Data**  
   - Realiza búsquedas específicas en la web para obtener información detallada o elementos relacionados con un tema en particular.  
   - Úsala para preguntas específicas o consultas sobre imágenes y detalles que no estén disponibles en bases de datos internas.  

8. **Search Data with Filters**  
    - Busca noticias relacionadas con un tema proporcionado. Opcionalmente, permite filtrar los resultados según un rango de fechas (fecha de inicio y fin).
    - Úsala cuando se requiera encontrar información o noticias específicas, ajustadas al contexto temporal si es necesario.
---

### Instrucciones importantes:
- **Formato de IDs de noticias:** Siempre utiliza arrays en el formato `['noticia_389', 'noticia_350', ...]`.  
- **Prioridad:** Si la pregunta es general, utiliza **ChromaDB Search** antes de **Wikipedia Search**.  
- **Cuenta de menciones:** Si la consulta es sobre cuántas veces se menciona a alguien, usa **Count Mentions**.  
- **Tema y nombres:** Para identificar el tema o nombres relevantes, usa **Extract Person Data**.  
- **Noticias relacionadas:** Si necesitas obtener noticias vinculadas a un usuario, utiliza **Get User News ChromaDB**.  
- **Imágenes:** Usa **Wikipedia Image** solo si se solicita específicamente una imagen.  

### Responde siempre:
1. En español.  
3. Seleccionando únicamente las herramientas necesarias según la pregunta.  
4. Evita combinar herramientas innecesariamente.

Cuando respondas, considera la prioridad y el contexto de la consulta para elegir las herramientas adecuadas.

"""
from langchain.memory import ConversationBufferMemory


prompt = PromptTemplate(input_variables=["tools", "query", "agent_scratchpad", "tool_names"], template=prompt_template)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    memory=memory
)



In [321]:
query = "¿Cuantas veces es mencionado “Juan Carlos Jobet”?"
response = agent.invoke(query)
print(response['output'])



> Entering new AgentExecutor chain...
Necesito contar cuántas veces se menciona "Juan Carlos Jobet" en un conjunto de noticias.
Action: count mentions
Action Input: ["Juan Carlos Jobet"]
Observation: 
Thought:Ahora necesito encontrar las noticias en las que aparece "Juan Carlos Jobet" para extraer información relevante.
Action: Get user news ChromaDB
Action Input: "Juan Carlos Jobet"
Observation: ['noticia_389', 'noticia_338', 'noticia_327', 'noticia_114']
Thought:Ahora debo procesar las noticias para extraer información personal relevante.
Action: Extract personal data from news
Action Input: ['noticia_389', 'noticia_338', 'noticia_327', 'noticia_114'], "Juan Carlos Jobet"
Observation: Texto 1:
Juan Carlos Jobet
Fecha de nacimiento: No se encontró información personal relevante.
Lugar de residencia: No se encontró información personal relevante.
Ocupación: Exministro de Energía y Minería durante el segundo gobierno de Sebastián Piñera.
Logros: Explicó el Comité Estratégico de Hidróg

In [322]:
query = "¿Quien es “Juan Carlos Jobet”?"
response = agent.invoke(query)
print(response['output'])



> Entering new AgentExecutor chain...
We can use Wikipedia Search to find general information about Juan Carlos Jobet.
Action: Wikipedia Search
Action Input: "Juan Carlos Jobet"
Observation: <p><b>Juan Carlos Jobet Eluchans</b> (Santiago, 14 de octubre de 1975) es un ingeniero comercial y político chileno. Se desempeñó como ministro de Energía desde junio de 2019, y como ministro de Minería desde diciembre de 2020, asumiendo simultáneamente ambos cargos como biministro de Estado de su país hasta marzo de 2022, bajo el segundo gobierno del presidente Sebastián Piñera. Entre julio de 2013 y marzo de 2014 ejerció como ministro del Trabajo y Previsión Social, durante el primer gobierno de Piñera.
</p>
Thought:We can also search for images related to Juan Carlos Jobet on Wikipedia to get a visual representation of him.
Action: Wikipedia image
Action Input: "Juan Carlos Jobet"
Observation: {'extract': '<p><b>Juan Carlos Jobet Eluchans</b> (Santiago, 14 de octubre de 1975) es un ingeniero c

In [323]:
query = " Juan Carlos Jobet, ¿Cuales son las declaraciones que ha realizado en el contexto del hidrógeno verde?"
response = agent.invoke(query)
print(response['output'])







> Entering new AgentExecutor chain...
Necesitamos encontrar las declaraciones específicas de Juan Carlos Jobet sobre el hidrógeno verde.
Action: Search data with filters
Action Input: "Declaraciones Juan Carlos Jobet hidrógeno verde"
Observation: {"tematica": "Declaraciones Juan Carlos Jobet hidr\u00f3geno verde", "start_date": "0001-01-01T00:00:00", "end_date": "9999-12-31T00:00:00", "news_id": ["noticia_114", "noticia_33", "noticia_34", "noticia_389", "noticia_327", "noticia_338", "noticia_136", "noticia_63", "noticia_188", "noticia_178"]}
Thought:Ahora necesitamos extraer las declaraciones específicas de Juan Carlos Jobet sobre el hidrógeno verde de las noticias obtenidas.
Action: Extract personal data from news
Action Input: ["noticia_114", "noticia_33", "noticia_34", "noticia_389", "noticia_327", "noticia_338", "noticia_136", "noticia_63", "noticia_188", "noticia_178"], "Juan Carlos Jobet"
Observation: Texto 1:
Nombre completo: Juan Carlos Jobet
Fecha de nacimiento: No se encont

In [324]:
query = "¿Cuál su opinión o su posición sobre el hidrógeno verde? de Juan Carlos Jobet"
response = agent.invoke(query)
print(response['output'])




> Entering new AgentExecutor chain...
Vamos a buscar información sobre la opinión o posición de Juan Carlos Jobet sobre el hidrógeno verde.
Action: Get user news ChromaDB
Action Input: "Juan Carlos Jobet"
Observation: ['noticia_389', 'noticia_338', 'noticia_327', 'noticia_114']
Thought:Ahora que tenemos las noticias en las que aparece Juan Carlos Jobet, vamos a extraer información personal relevante de esas noticias.
Action: Extract personal data from news
Action Input: ['noticia_389', 'noticia_338', 'noticia_327', 'noticia_114'], "Juan Carlos Jobet"
Observation: Texto 1:
Nombre completo: Juan Carlos Jobet
Fecha de nacimiento: No se encontró información personal relevante.
Lugar de residencia: No se encontró información personal relevante.
Ocupación: Exministro de Energía y Minería.
Logros: Explicó el Comité Estratégico de Hidrógeno Verde y busca establecer una hoja de ruta para la industria sostenible del hidrógeno verde.
Citas textuales relevantes: 
1. "Hidrógeno verde puede ser má

In [325]:
query = "Dame el enlace de una foto de Juan Carlos Jobet "
response = agent.invoke(query)
print(response['output'])



> Entering new AgentExecutor chain...
Necesito buscar una imagen de Juan Carlos Jobet en Wikipedia.
Action: Wikipedia image
Action Input: "Juan Carlos Jobet"
Observation: {'extract': '<p><b>Juan Carlos Jobet Eluchans</b> (Santiago, 14 de octubre de 1975) es un ingeniero comercial y político chileno. Se desempeñó como ministro de Energía desde junio de 2019, y como ministro de Minería desde diciembre de 2020, asumiendo simultáneamente ambos cargos como biministro de Estado de su país hasta marzo de 2022, bajo el segundo gobierno del presidente Sebastián Piñera. Entre julio de 2013 y marzo de 2014 ejerció como ministro del Trabajo y Previsión Social, durante el primer gobierno de Piñera.\n</p>', 'image_url': 'https://upload.wikimedia.org/wikipedia/commons/6/6e/JUAN_CARLOS_JOBET.jpg'}
Thought:He encontrado la información y la imagen de Juan Carlos Jobet.
Final Answer: La foto de Juan Carlos Jobet se puede encontrar en el siguiente enlace: https://upload.wikimedia.org/wikipedia/commons/6

In [327]:
query = "Dame una foto de messi"
response = agent.invoke(query)
print(response['output'])



> Entering new AgentExecutor chain...
Necesitamos buscar una imagen relacionada con Lionel Messi en Wikipedia.
Action: Wikipedia image
Action Input: "Lionel Messi"
Observation: {'extract': '<p><b>Lionel Andrés Messi Cuccittini</b> (Rosario, 24 de junio de 1987), conocido como <b>Leo Messi</b>, es un futbolista argentino que juega como delantero o centrocampista. Desde 2023, integra el plantel del Inter Miami de la MLS canadoestadounidense. Es también internacional con la selección de Argentina, de la que es capitán.\n</p><p>Con el Fútbol Club Barcelona, al que estuvo ligado más de veinte años, ganó 35 títulos, entre ellos, diez de La Liga, cuatro de la Liga de Campeones de la UEFA y siete de la Copa del Rey.\n</p><p>Considerado con frecuencia el mejor jugador del mundo y uno de los mejores de todos los tiempos,<sup id="cite_ref-11" class="reference separada"><span>[</span>10<span>]</span></sup>\u200b es el único en la historia que ha ganado, entre otras distinciones, ocho veces el Ba

In [328]:
query = "Quienes son las personas que mas se mencionan entre junio del 2023 y mayo del 2024 con tematica de hidrogeno verde?"
response = agent.invoke(query)
print(response['output'])



> Entering new AgentExecutor chain...
Necesito encontrar las personas más mencionadas en noticias sobre hidrógeno verde en un rango de fechas específico.
Action: Search data with filters
Action Input: tema="hidrogeno verde", fecha_inicio="junio 2023", fecha_fin="mayo 2024"
Observation: {"tematica": "hidrogeno verde", "start_date": "2023-06-01T00:00:00", "end_date": "2024-05-31T00:00:00", "news_id": ["noticia_127", "noticia_34", "noticia_33"]}
Thought:Ahora que tengo las noticias relevantes sobre hidrógeno verde en el rango de fechas especificado, debo contar las menciones de personas en esas noticias.
Action: count mentions
Action Input: ["noticia_127", "noticia_34", "noticia_33"]
Observation: Gabriel Boric, 3 veces; Eliana Ximena Oyarzún Nancuante “Les, 1 veces; Presidente, 1 veces; Sebastián Piñera, 3 veces; Juan Carlos Jobet, 4 veces; Presidenta Michelle Bachelet, 3 veces; Rosa Devés, 1 veces; Jorge Flies, 3 veces; Diego Pardow, 1 veces; Jobet, 1 veces; Presidente Boric, 1 veces; 